In [1]:
import re
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
# ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
ADDRESS2 = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
ADDRESS3 = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-10-25 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-11-28 08:00"
# OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
OUTPUT_FILE2 = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2"
OUTPUT_FILE3 = r"C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3"

API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))

In [2]:
fetch_and_save_erc20_transfers(
        address=ADDRESS2,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE2,
        offset=1000,
    )

transfers_df2 = pd.read_csv(OUTPUT_FILE2 + "/erc20_transfers.csv")
record_df_1_2, _, _ = process_transactions(transfers_df2, OUTPUT_FILE2, ADDRESS2, base_pattern)
fetch_and_save_erc20_transfers(
        address=ADDRESS3,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE3,
        offset=1000,
    )
transfers_df3 = pd.read_csv(OUTPUT_FILE3 + "/erc20_transfers.csv")
record_df_1_3, _, _  = process_transactions(transfers_df3, OUTPUT_FILE3, ADDRESS3, base_pattern)

2025-01-20 12:03:40,380 - INFO - Start block: 21038905, End block: 21282523
2025-01-20 12:03:40,381 - INFO - Fetching ERC20 transfers from block 21038905 to 21282523...
2025-01-20 12:03:42,400 - INFO - Page 1: Retrieved 596 transactions.
2025-01-20 12:03:44,572 - WARNING - API request failed on page 2: Request failed: No transactions found
2025-01-20 12:03:44,594 - INFO - Final dataset contains 596 unique transactions.
2025-01-20 12:03:44,937 - INFO - Data successfully saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2\erc20_transfers.csv
2025-01-20 12:03:44,943 - INFO - Transfers saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address2


...Merging 212 groups of transactions...
Transaction Hash 0x2b198f8b99cc4a183fdb5125b09e1f32521b7f5351e6b12307d5022ba030708c contains 4 records:
Transaction Hash 0xe7965c51c772e31d0195646c27b1eda26ac069483f2e49db737cc8cb94fd3369 contains 4 records:
Transaction Hash 0xeb47a72d88c47d28a4150b339cab0bf693cc39a360f021743785a94102be0ddd contains 6 records:


2025-01-20 12:03:55,328 - INFO - Start block: 21038905, End block: 21282523
2025-01-20 12:03:55,328 - INFO - Fetching ERC20 transfers from block 21038905 to 21282523...
2025-01-20 12:03:58,239 - INFO - Page 1: Retrieved 1000 transactions.
2025-01-20 12:04:00,089 - INFO - Page 2: Retrieved 1000 transactions.
2025-01-20 12:04:02,040 - INFO - Page 3: Retrieved 474 transactions.
2025-01-20 12:04:03,214 - WARNING - API request failed on page 4: Request failed: No transactions found
2025-01-20 12:04:03,273 - INFO - Final dataset contains 2474 unique transactions.
2025-01-20 12:04:03,692 - INFO - Data successfully saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3\erc20_transfers.csv
2025-01-20 12:04:03,700 - INFO - Transfers saved to C:\Users\HUAWEI\Documents\GitHub\Project\etherscan\address3


...Merging 918 groups of transactions...
Transaction Hash 0x052ca792656135f686258d095a1cccb03c9b1d8ad6110a890bb84ad8c7e18d13 contains 4 records:
Transaction Hash 0x06b5ecfb93d25c3d81ebeee11cfd41d1647e8b5aa9b1f3ccd7a435996784ea61 contains 4 records:
Transaction Hash 0x09c275b15ecc39e71b98e336bcef59546d008bd602fb673f8de53415063283b5 contains 4 records:
Transaction Hash 0x0e6e0c9402d39ffed6e0fac563abc8a1990e34f50296958b0f4d2983c817681d contains 4 records:
Transaction Hash 0x1338ed70de802a70967c6ab2bad8b254c001f3d80680242680dcc07aeb68ded4 contains 4 records:
Transaction Hash 0x1437c9cf8bd31aa1aef8e427c918e513f143258ad7425c935086f39b6082078a contains 10 records:
Transaction Hash 0x22d64dc7cd02361e023c8aa05b7cd761cb483ab4e66f87234240cf2ccb77162d contains 8 records:
Transaction Hash 0x230939d9187c3a932b7a077bd6a9927dfd8e6530c5344fd82f3d4715a0a7c9b2 contains 4 records:
Transaction Hash 0x2cf69cdc1723babb332544cb41e33988e87b678be193d453ef2457d34e1e3b96 contains 6 records:
Transaction Hash 0x3a8

In [ ]:
def assign_band(date):
    """
    Assign a band identifier based on the date.
    """
    if pd.to_datetime('2024-04-25') <= date <= pd.to_datetime('2024-05-21'):
        return "First Band"
    elif pd.to_datetime('2024-08-12') <= date <= pd.to_datetime('2024-09-02'):
        return "Second Band"
    elif pd.to_datetime('2024-09-04') <= date <= pd.to_datetime('2024-09-04'):
        return "Third Band"
    elif pd.to_datetime('2024-09-06') <= date <= pd.to_datetime('2024-09-06'):
        return "Fourth Band"
    elif pd.to_datetime('2024-09-06') <= date <= pd.to_datetime('2024-09-06'):
        return "Fifth Band"
    elif pd.to_datetime('2024-09-06') <= date <= pd.to_datetime('2024-09-07'):
        return "Sixth Band"
    elif pd.to_datetime('2024-09-16') <= date <= pd.to_datetime('2024-09-16'):
        return "Seventh Band"
    elif pd.to_datetime('2024-09-17') <= date <= pd.to_datetime('2024-09-17'):
        return "Eighth Band"
    elif pd.to_datetime('2024-09-18') <= date <= pd.to_datetime('2024-09-19'):
        return "Ninth Band"
    elif pd.to_datetime('2024-09-26') <= date <= pd.to_datetime('2024-09-26'):
        return "Tenth Band"
    elif pd.to_datetime('2024-09-28') <= date <= pd.to_datetime('2024-09-28'):
        return "Eleventh Band"
    elif pd.to_datetime('2024-09-29') <= date <= pd.to_datetime('2024-10-24'):
        return "Twelfth Band"
    elif pd.to_datetime('2024-10-25') <= date <= pd.to_datetime('2024-10-28'):
        return "Thirteenth Band"
    elif pd.to_datetime('2024-11-01') <= date <= pd.to_datetime('2024-11-06'):
        return "Fourteenth Band"
    elif pd.to_datetime('2024-11-08') <= date <= pd.to_datetime('2024-11-09'):
        return "Fifteenth Band"
    elif pd.to_datetime('2024-11-11') <= date <= pd.to_datetime('2024-11-11'):
        return "Sixteenth Band"
    elif pd.to_datetime('2024-11-12') <= date <= pd.to_datetime('2024-11-19'):
        return "Seventeenth Band"
    elif pd.to_datetime('2024-11-19') <= date <= pd.to_datetime('2024-11-21'):
        return "Eighteenth Band"
    elif pd.to_datetime('2024-12-02') <= date <= pd.to_datetime('2024-12-04'):
        return "Nineteenth Band"
    elif pd.to_datetime('2024-12-09') <= date <= pd.to_datetime('2024-12-12'):
        return "Twentieth Band"
    elif pd.to_datetime('2024-12-20') <= date <= pd.to_datetime('2024-12-30'):
        return "Twenty-first Band"
    elif pd.to_datetime('2024-12-31') <= date <= pd.to_datetime('2024-12-31'):
        return "Twenty-second Band"
    elif pd.to_datetime('2025-01-01') <= date <= pd.to_datetime('2025-01-02'):
        return "Twenty-third Band"
    elif pd.to_datetime('2025-01-02') <= date <= pd.to_datetime('2025-01-09'):
        return "Twenty-fourth Band"
    elif pd.to_datetime('2025-01-09') <= date <= pd.to_datetime('2025-01-17'):
        return "Twenty-fifth Band"
    else:
        return "Unassigned Band"

# 计算pnl
def process_eth_pnl(record_df):
    if not record_df.empty:
        eth_record_df = record_df[record_df["formatted_record"].str.contains(r'ETH', regex=True, case=False)]
        
        if eth_record_df.empty:
            print("No ETH-related transactions found.")
            return pd.DataFrame()
        eth_pnl_df = calculate_pnl(eth_record_df).sort_values("timeStamp")
        print(tabulate(eth_pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
        eth_pnl_df.to_csv("eth_pnl_details.csv", index=False)
        # 合并所有 PnL 数据
        return eth_pnl_df
    
    else:
        print("No transactions found")
        return pd.DataFrame()


combined_record_df = pd.concat([record_df_1_2, record_df_1_3], ignore_index=True)
eth_record_df = process_eth_pnl(combined_record_df)

+---------------------+-------------+--------+----------+----------+---------------+----------------+
| Date                |   timeStamp | Type   |   amount |    price |           pnl |   net_position |
+=====================+=============+========+==========+==========+===============+================+
| 2024-10-31 14:22:47 |  1730384567 | SELL   |  1117.94 |  2575.91 | -106454490.73 |       -1117.94 |
+---------------------+-------------+--------+----------+----------+---------------+----------------+
| 2024-10-31 14:28:59 |  1730384939 | BUY    |   224.21 |  2568.70 |   21352271.35 |        -893.72 |
+---------------------+-------------+--------+----------+----------+---------------+----------------+
| 2024-10-31 14:30:47 |  1730385047 | BUY    |   896.31 |  2570.26 |   85355823.21 |           2.59 |
+---------------------+-------------+--------+----------+----------+---------------+----------------+
| 2024-10-31 14:35:47 |  1730385347 | SELL   |   112.05 |  2567.99 |  -10671029.77

In [16]:
print(eth_record_df.head(1))

                 Date   timeStamp  Type       amount        price  \
0 2024-10-31 14:22:47  1730384567  SELL  1117.936493  2575.905068   

            pnl  net_position  
0 -1.064545e+08  -1117.936493  


In [19]:
def reproduce_band_table(record_df):
    if record_df.empty:
        print("No transactions found.")
        return
    # Add band identifier
    record_df['Band'] = record_df['Date'].apply(assign_band)

    # Group by band and calculate summary metrics
    band_summary = record_df.groupby('Band').agg(
        ETH_Amount=('amount', 'sum'),  # Sum of ETH amounts
        Band_Value=('amount', lambda x: (x * record_df.loc[x.index, 'price']).sum() / 1e6),  # Convert to million USD
        Open_Price=('price', 'first'),  # First price in the band
        Close_Price=('price', 'last'),  # Last price in the band
        PnL=('pnl', 'sum')  # Sum of PnL in the band
    ).reset_index()

    # Print band summary
    print("Band-wise PnL Summary:")
    print(tabulate(band_summary, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))

    # Save band summary to CSV
    band_summary.to_csv("band_pnl_summary.csv", index=False)
    print("Band-wise PnL summary saved to band_pnl_summary.csv")

reproduce_band_table(eth_record_df)

Band-wise PnL Summary:
+------------------+--------------+--------------+--------------+---------------+----------------+
| Band             |   ETH_Amount |   Band_Value |   Open_Price |   Close_Price |            PnL |
+==================+==============+==============+==============+===============+================+
| Eighteenth Band  |     11483.61 |        35.82 |      3120.00 |       3108.34 |  1087281691.91 |
+------------------+--------------+--------------+--------------+---------------+----------------+
| Fifteenth Band   |     12222.55 |        35.70 |      2906.94 |       3033.01 |  1159665724.62 |
+------------------+--------------+--------------+--------------+---------------+----------------+
| Fourteenth Band  |      1125.42 |         2.73 |      2430.69 |       2425.97 |  -107331738.83 |
+------------------+--------------+--------------+--------------+---------------+----------------+
| Seventeenth Band |     23421.76 |        76.00 |      3340.13 |       3180.00 |   -8